In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from skimage.draw import disk, polygon
from glob import glob
import cv2

import                              torch, copy
import                              torch.nn as nn

from config                         import *
from diffusers                      import (AutoencoderKL, DDPMScheduler, UNet2DModel, DDIMScheduler)
# from source_unet_2d                 import UNet2DModel
from modeling_utils                 import *
from peft                           import get_peft_model, LoraConfig, TaskType

# root_dir = 'C:/Users/Talha/OneDrive - Higher Education Commission/Documents/GitHub/Diffusion-Codes/Diffusers-Testing'
# os.chdir(root_dir)

/home/ee/anaconda3/envs/TaN2/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
!python uncertainty_analysis.py

Metrics saved to C:/Users/Talha/OneDrive - Higher Education Commission/Documents/GitHub/Diffusion-Codes/Diffusers-Testing/testing_analysis\uncertainty_metrics.csv
✅ Uncertainty metrics saved to C:/Users/Talha/OneDrive - Higher Education Commission/Documents/GitHub/Diffusion-Codes/Diffusers-Testing/testing_analysis\uncertainty_metrics.csv


2025-05-23 19:32:07.875131: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-23 19:32:09.852653: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [3]:
csv_path = 'testing_analysis/uncertainty_metrics.csv'

pd.read_csv(csv_path)

,Patient_ID,GT_Components,Pred_Components,Component_Diff,GT_Smoothness,Pred_Smoothness,Smoothness_Diff
0,0,1,1,0,"([1.0765577989436452], 1.0765577989436452)","([1.0765577989436452], 1.0765577989436452)",0.0
1,1,2,1,1,"([1.0700973119248192, 1.0570850420432631], 1.0...","([1.0700973119248192, nan], nan)",NaN
2,2,1,1,0,"([1.3657568924106536], 1.3657568924106536)","([1.3657568924106536], 1.3657568924106536)",0.0
3,3,2,1,1,"([1.3657568924106536, 1.075845765087769], 1.22...","([1.075845765087769, nan], nan)",NaN


In [2]:
batch_size  = 2
device      = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
noise_pred  = torch.randn(batch_size, 4, 32, 32).to(device)
scheduler   = DDPMScheduler(num_train_timesteps=1000, beta_start = BETA_START, beta_end = BETA_END, beta_schedule = NOISE_SCHEDULER)
t           = torch.randint(0, scheduler.config.num_train_timesteps, (noise_pred.size(0),), device=device).long()
zt          = torch.randn(batch_size, 4, 32, 32).to(device)


In [6]:
def denoise_and_decode_in_one_step(batch_size, noise_pred, timesteps, zt, scheduler, device, inference = False):
    
    """
    Denoise and decode the latent zt to obtain the predicted mask.
    """
    z0_hat_list   = []
        
    if inference:
        noise_pred = noise_pred.to(device = 'cpu') 
        timesteps  = timesteps.to(device = 'cpu') 
        zt         = zt.to(device = 'cpu')
        
    for batch_idx in range(noise_pred.shape[0]):
        # z0_hat = (zt - ((1 - scheduler.alphas_cumprod).sqrt() * noise_pred)) / scheduler.alphas_cumprod.sqrt()
        # z0_hat   = scheduler.step(noise_pred[batch_idx].unsqueeze(0), timesteps[batch_idx].unsqueeze(0), zt[batch_idx].unsqueeze(0)).pred_original_sample
        alpha_t                     = scheduler.alphas_cumprod[timesteps[batch_idx].item()]
        z0_hat                      = (zt[batch_idx].unsqueeze(0) - (1 - alpha_t).sqrt() * noise_pred[batch_idx].unsqueeze(0)) / alpha_t.sqrt()
        z0_hat_list.append(z0_hat)
    
    z0_hat   = torch.cat(z0_hat_list,   dim = 0) # (B, 4, 32, 32)
    
    return z0_hat


In [4]:
z0_hat = denoise_and_decode_in_one_step(batch_size, noise_pred, t, zt, scheduler, device, inference = True)

In [7]:
z0_hat1 = denoise_and_decode_in_one_step(batch_size, noise_pred, t, zt, scheduler, device, inference = False)

In [8]:
z0_hat1.shape

torch.Size([2, 4, 32, 32])

In [10]:
torch.equal(z0_hat.to(device), z0_hat1.to(device))

False